In [1]:
!pip install -q accelerate==0.21.0 peft==0.9.0 bitsandbytes==0.40.2 transformers==4.31.0 trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 263.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np

data = pd.read_json("Dataset for DS Case Study.json",lines=True)

def format_instruction(reviewText: str, summary: str):
    return f"""### Instruction:
Recommend Product From The User Data.

### Input:
{reviewText.strip()}

### Summary:
{summary}
""".strip()


def generate_instruction_dataset(data_point):
    return {
        "reviewText": data_point["reviewText"],
        "summary": data_point["summary"],
        "text": format_instruction(data_point["reviewText"],data_point["summary"])
    }

def process_dataset(data: pd.DataFrame):
    data = data.apply(generate_instruction_dataset, axis=1, result_type='expand')
    columns_to_drop = ['reviewerID', 'asin','reviewerName','helpful','overall','unixReviewTime','reviewTime']
    data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])
    return data

## APPLYING PREPROCESSING ON WHOLE DATASET
data_train = process_dataset(data[:5000])
data_test = process_dataset(data[5000:7000])
data_validation = process_dataset(data[7000:8000])

# Shuffle the data
data_train = data_train.sample(frac=1).reset_index(drop=True)
data_test = data_test.sample(frac=1).reset_index(drop=True)
data_validation = data_validation.sample(frac=1).reset_index(drop=True)

# Select 1000 rows from the training split
train_data = data_train[:1000]

# Select 100 rows from the test and validation splits
test_data = data_test[:500]
validation_data = data_validation[:50]

train_data,test_data,validation_data

(                                            reviewText  \
 0    I use this on auto repairs to ensure a locked ...   
 1    This nozzle is built like a tank and will last...   
 2    I have been using Anco wiper blades for all of...   
 3    This air deflector was installed beneath our c...   
 4    Very good quality, light weight and built to l...   
 ..                                                 ...   
 995  Use with mitten and warm to hot water followin...   
 996  These fit perfectly and are easy to use and ke...   
 997  There are plenty of RV Park type campgrounds o...   
 998  Installed these on my 2002 Jeep TJ and really ...   
 999  I had purchased two of these drying towels as ...   
 
                                        summary  \
 0              the standard for thread locking   
 1                    Last Nozzle I'll ever buy   
 2                             Anco Is The Best   
 3                        Premium Air Deflector   
 4                       Can't Rock

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id =  "NousResearch/Llama-2-7b-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [4]:
index=0
dialogue = data_test['reviewText'][index]
summary = data_test['summary'][index]

prompt = f"""
Recommend Product From The User Data.

### Input:
{dialogue}

### Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=100,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Recommend Product From The User Data.

### Input:
Even in NJ weather that can be sleet, snow or rain at any time, these perform and are quiet. Now my friends want me to order them for their cars.

### Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Works great

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:

Recommend Product From The User Data.

### Input:
Even in NJ weather that can be sleet, snow or rain at any time, these perform and are quiet. Now my friends want me to order them for their cars.

### Summary:


### Output:


### Author:


### Date:


### Version:


### Requirement:


### Test:


### Note:


### Reference:


### Link:


### Example:


### Example:


### Example:


### Example:


### Exa

In [5]:

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():

        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [6]:

from peft import prepare_model_for_int8_training

model.gradient_checkpointing_enable()
model = prepare_model_for_int8_training(model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:145: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [7]:

print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [8]:

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    # target_modules=["query_key_value"],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], #specific to Llama models.
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 16777216 || all params: 3517190144 || trainable%: 0.477006226934315


In [9]:

OUTPUT_DIR = "llama2-docsum-adapter"

In [10]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=5,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [11]:
import datasets

# Assuming train_data and validation_data are lists of data
train_dataset = datasets.Dataset.from_pandas(train_data)
validation_dataset = datasets.Dataset.from_pandas(validation_data)

from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,  # Pass the dataset object instead of the list
    eval_dataset=validation_dataset,  # Pass the dataset object instead of the list
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)
trainer.neftune_noise_alpha = None
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
62,1.809200,1.957631
124,1.999400,1.949732
186,1.879400,1.967961
248,1.506500,1.986978
310,1.173600,1.993182


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downlo

TrainOutput(global_step=310, training_loss=1.7694966843051294, metrics={'train_runtime': 3768.2548, 'train_samples_per_second': 1.327, 'train_steps_per_second': 0.082, 'total_flos': 1.5333345247002624e+16, 'train_loss': 1.7694966843051294, 'epoch': 4.96})

In [12]:
train_data.head()

,reviewText,summary,text
0,I use this on auto repairs to ensure a locked ...,the standard for thread locking,### Instruction:\nRecommend Product From The U...
1,This nozzle is built like a tank and will last...,Last Nozzle I'll ever buy,### Instruction:\nRecommend Product From The U...
2,I have been using Anco wiper blades for all of...,Anco Is The Best,### Instruction:\nRecommend Product From The U...
3,This air deflector was installed beneath our c...,Premium Air Deflector,### Instruction:\nRecommend Product From The U...
4,"Very good quality, light weight and built to l...",Can't Rock and Roll...,### Instruction:\nRecommend Product From The U...


In [13]:

peft_model_path="./peft-recommend-summary"

trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./peft-recommend-summary/tokenizer_config.json',
 './peft-recommend-summary/special_tokens_map.json',
 './peft-recommend-summary/tokenizer.model',
 './peft-recommend-summary/added_tokens.json',
 './peft-recommend-summary/tokenizer.json')

In [14]:
from transformers import TextStreamer
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer

In [15]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

peft_model_dir = "peft-recommend-summary"

# load base LLM model and tokenizer
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
index=14

dialogue = data_test['reviewText'][index]
summary = data_test['summary'][index]

prompt = f"""
Recommend Product From The User Data.

### Input:
{dialogue}

### Summary:
"""

input_ids = tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()
# output = tokenizer.decode(
#     trained_model.generate(
#         inputs_ids,
#         max_new_tokens=100,
#     )[0],
#     skip_special_tokens=True
# )
outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, )
output= tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'TRAINED MODEL GENERATED TEXT :\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Recommend Product From The User Data.

### Input:
These work great to keep the stuff on the shelves in the refer or in the cabinets.  A must buy.

### Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Holder in cabinets or refer

---------------------------------------------------------------------------------------------------
TRAINED MODEL GENERATED TEXT :
Great product.  Easy to use.  Sturdy.  Does the job.  I would buy again.

### Details:
These work great to keep the stuff on the shelves in the refer or in the cabinets.  A must buy.

### Summary:
Great product.  Easy to use.  Sturdy.  Does the job.  I would buy again.

### Details:


In [23]:
!pip install kora

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [25]:
!zip -r '/content/model.zip' '/content/peft-recommend-summary'

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [26]:
import subprocess

subprocess.run(['zip', '-r', '/content/model.zip', '/content/peft-recommend-summary'])

CompletedProcess(args=['zip', '-r', '/content/model.zip', '/content/peft-recommend-summary'], returncode=0)

In [28]:
import subprocess

subprocess.run(['zip', '-r', '/content/model1_llama2-docsum-adapter.zip', '/content/llama2-docsum-adapter'])

CompletedProcess(args=['zip', '-r', '/content/model1_llama2-docsum-adapter.zip', '/content/llama2-docsum-adapter'], returncode=0)